# Looking at Inference on full datasets
## 10/03/2020

Trained model by fusing depth from res2,4, not like in first model with res2,3,4,5. Loss looks very similar for both, double checking only those two feature maps fuse.

In [1]:
config_file = '../configs/deepent_fuse_rcnn_R_50_FPN.yaml'
threshold = 0.5
model = '../output/fuse_24/model_final.pth'
opts = []

class Args:
    def __init__(self, conf, mod, opt):
        self.config_file = conf
        self.model = mod
        self.opts = []
        self.output = None
        
args = Args(config_file, model, opts)

In [2]:
from detectron2.config import get_cfg
from deepent.config import add_deepent_config
from detectron2.engine import default_setup

cfg = get_cfg()
add_deepent_config(cfg)
cfg.merge_from_file(args.config_file)
cfg.merge_from_list(args.opts)
cfg.freeze()
default_setup(cfg, args)

[03/14 04:35:56 detectron2]: Rank of current process: 0. World size: 1
[03/14 04:35:56 detectron2]: Environment info:
------------------------  -------------------------------------------------------------------
sys.platform              linux
Python                    3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) [GCC 7.2.0]
Numpy                     1.15.4
Detectron2 Compiler       GCC 5.4
Detectron2 CUDA Compiler  10.1
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.3.1
PyTorch Debug Build       False
torchvision               0.4.2
CUDA available            True
GPU 0                     Tesla T4
CUDA_HOME                 /usr/local/cuda-10.1
NVCC                      Cuda compilation tools, release 10.1, V10.1.243
Pillow                    6.2.2
cv2                       4.1.2
------------------------  -------------------------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - Intel(R) Math Kernel Library Version 2018.0.3 Produ

In [3]:
from deepent.data.register_datasets import register_datasets
register_datasets(f'/home/ubuntu/RGBD-Tree-Segs-Clean/')

Registering Kelowna_train
Registering AMECT9_train
Registering SALCT1_train
Registering CPT2b_train
Registering CPT2a-n_train
Registering Kelowna_test
Registering AMECT9_test
Registering SALCT1_test
Registering CPT2b_test
Registering CPT2a-n_test


In [5]:
from detectron2.data import MetadataCatalog, DatasetCatalog
import random

data = list(DatasetCatalog.get("CPT2b_test"))
metadata = MetadataCatalog.get("CPT2b_test")

In [3]:
from deepent.modelling.backbone.deepent_fused_resnet import build_deepent_fpn_backbone
from detectron2.layers import ShapeSpec

input_shape = ShapeSpec(channels=len(cfg.MODEL.PIXEL_MEAN))
dd = build_deepent_fpn_backbone(cfg,input_shape).eval().cuda()

In [6]:
dic = random.sample(data, 1)

In [7]:
import cv2
import torch

rgba = cv2.imread(dic[0]["file_name"], cv2.IMREAD_UNCHANGED)
rgb, d = rgba[:,:, :3], rgba[:,:, 3:]

In [18]:
d = torch.tensor(rgba,dtype=torch.float32).permute(-1,0,1).unsqueeze(0).cuda()

In [19]:
d.shape

torch.Size([1, 4, 640, 640])

In [20]:
out = dd(d)

In [21]:
out.keys()

dict_keys(['p2', 'p3', 'p4', 'p5', 'p6'])

The model is fine. Lets compare results on full ortho of CPT2a-n, for rgbd_long (i.e. fusing all depth channels) and fuse_24 (fusing only the aforementioned 2,4). Both models running inference are final at 120000 iterations.